# DuckDBManager example usages

## Initializing

In [1]:
import duckdb_manager as ddb

db_manager = ddb.DuckDBManager()

### Note: If the program can't find the duck db file, it will automaticlally recreate the database using the 'import_all_csv_files'. 

### This functionality will not work if you don't have the necessary csv files, so if for whatever reason you don't have the data, you can just run the 'populate_database' notebook file to get the data. 


## Exporting

### Since github does not allow users to upload files larger than 100mb, the duckdb database has to split into seperate csv files. However, I don't recommend using the CSV files directly as pandas dataframes since performance becomes an issue whenever CSV files are too large.

In [2]:
db_manager.export_to_csv()

Exporting DataEntry CSV files: 100%|██████████| 12/12 [00:54<00:00,  4.55s/it]


### The 'check_row_length' function takes in a string and spits out the number of rows in the table specified.

In [2]:
print(f"DataEntry number of rows: {db_manager.check_row_length('DataEntry')}")
print(f"DimZipCode number of rows: {db_manager.check_row_length('DimZipCode')}")
print(f"DimYear number of rows: {db_manager.check_row_length('DimYear')}")
print(f"DimNaics number of rows: {db_manager.check_row_length('DimNaics')}")

DataEntry number of rows: 36216155
DimZipCode number of rows: 39331
DimYear number of rows: 12
DimNaics number of rows: 2216


# Export to nested zip folders

In [3]:
!ls database

exported_tables         nested_zip              us_economic_data.duckdb


In [ ]:
import dataexporter as dex
import os
export_dir='database/nested_zip'
db_path='database/us_economic_data.duckdb'
db_exporter = dex.DataExporter(export_dir=export_dir, db_path=db_path)
db_exporter.export_geo_nested_csv()

### The 'get_schema' gives us the database schema

In [3]:
db_manager.get_schema()

{'DataEntry': [('EntryID', 'INTEGER'),
  ('GeoID', 'VARCHAR'),
  ('NaicsCode', 'VARCHAR'),
  ('Year', 'INTEGER'),
  ('Establishments', 'INTEGER'),
  ('Employees', 'INTEGER'),
  ('Payroll', 'INTEGER'),
  ('IndustryLevel', 'INTEGER')],
 'DimNaics': [('NaicsCode', 'VARCHAR'), ('industry_detail', 'VARCHAR')],
 'DimYear': [('Year', 'INTEGER'), ('YearDescription', 'VARCHAR')],
 'DimZipCode': [('GeoID', 'VARCHAR'),
  ('City', 'VARCHAR'),
  ('State', 'VARCHAR')]}

### The 'execute_query' function allows you to create a custom query and get a dataframe back.

In [4]:
query = """ 
    SELECT 
        *
    FROM
        DimZipCode
    LIMIT 10
"""
db_manager.execute_query(query)

,GeoID,City,State
0,86433,Oatman,AZ
1,25161,Powellton,WV
2,67640,Gorham,KS
3,37371,Athens,TN
4,56134,Hardwick,MN
5,16427,Mill Village,PA
6,52362,Wyoming,IA
7,28746,Lake Lure,NC
8,23407,Mappsville,VA
9,76021,Bedford,TX


In [7]:
db_manager.filter_by_year_zip_industry(zip_prefix=0, industry_level=2, year=2021)

,EntryID,GeoID,NaicsCode,Year,Establishments,Employees,Payroll,IndustryLevel
0,33991666,01007,11,2021,3,0,0,2
1,33991667,01451,11,2021,4,0,0,2
2,33991668,01473,11,2021,5,0,0,2
3,33991669,01720,11,2021,3,0,0,2
4,33991670,01770,11,2021,3,0,0,2
...,...,...,...,...,...,...,...,...
21745,35596577,08540,99,2021,3,0,0,2
21746,35596578,08618,99,2021,4,0,0,2
21747,35596579,08701,99,2021,10,0,0,2
21748,35596580,08831,99,2021,3,0,0,2


In [6]:
query = """ 
    SELECT 
        *
    FROM
        DataEntry
    WHERE
        NaicsCode = '00'
    LIMIT 10
"""
db_manager.execute_query(query)

,EntryID,GeoID,NaicsCode,Year,Establishments,Employees,Payroll,IndustryLevel
0,35597371,91327,00,2012,6,0,0,2
1,35597372,91337,00,2012,3,0,200,2
2,35597373,91603,00,2012,4,0,186,2
3,35597374,91324,00,2012,1423,15910,479071,2
4,35597375,91326,00,2012,484,4053,116138,2
5,35597376,91329,00,2012,3,0,0,2
6,35597377,91331,00,2012,832,12004,473834,2
7,35597378,91333,00,2012,2,0,0,2
8,35597379,91340,00,2012,665,10172,391684,2
9,35597380,91322,00,2012,12,0,1600,2


In [ ]:
import duckdb
import os
import csv

# Connect to DuckDB
conn = duckdb.connect(database='database/us_economic_data.duckdb', read_only=False)
cursor = conn.cursor()

# Fetch unique GeoIDs
cursor.execute("SELECT DISTINCT GeoID FROM DataEntry")
geo_ids = cursor.fetchall()

base_dir = "/nested_zip"
os.makedirs(base_dir, exist_ok=True)
# Create main directories for each GeoID
for (geo_id,) in geo_ids:
    geo_path = os.path.join(base_dir, geo_id)
    os.makedirs(geo_path, exist_ok=True)
    
    # Fetch unique combinations of Year and IndustryLevel for each GeoID
    cursor.execute("""
    SELECT DISTINCT Year, IndustryLevel
    FROM DataEntry
    WHERE GeoID = ?
    """, (geo_id,))
    combinations = cursor.fetchall()

    # Create CSV files for each combination of Year and IndustryLevel within the GeoID directory
    for year, industry_level in combinations:
        csv_file_name = f"data_{year}_{industry_level}.csv"
        csv_file_path = os.path.join(geo_path, csv_file_name)

        # Fetch data for each combination
        cursor.execute("""
        SELECT GeoID, Establishments, "Employees", "Payroll" FROM DataEntry
        WHERE GeoID = ? AND Year = ? AND IndustryLevel = ?
        """, (geo_id, year, industry_level))
        data_entries = cursor.fetchall()

        # Write data to the CSV file
        with open(csv_file_path, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["Zip", "Establishments", "Employees", "Payroll"])
            writer.writerows(data_entries)

# Close the database connection
cursor.close()
conn.close()


In [3]:
import duckdb
import os
import csv

# Connect to DuckDB
conn = duckdb.connect(database='database/us_economic_data.duckdb', read_only=False)
cursor = conn.cursor()

# Define the base directory for nested ZIP directories
base_dir = "./nested_zip"
os.makedirs(base_dir, exist_ok=True)  # Ensure the base directory exists

# Fetch unique GeoIDs
cursor.execute("SELECT DISTINCT GeoID FROM DataEntry")
geo_ids = cursor.fetchall()

# Create nested directories for each digit of each GeoID
for (geo_id,) in geo_ids:
    # Create a path by nesting directories for each digit of the GeoID
    path_components = [base_dir] + list(geo_id)
    geo_path = os.path.join(*path_components)
    os.makedirs(geo_path, exist_ok=True)
    
    # Fetch unique combinations of Year and IndustryLevel for this GeoID
    cursor.execute("""
        SELECT DISTINCT Year, IndustryLevel FROM DataEntry
        WHERE GeoID = ?
    """, (geo_id,))
    combinations = cursor.fetchall()

    # Create CSV files for each combination of Year and IndustryLevel in the deepest directory
    for year, industry_level in combinations:
        csv_file_name = f"data_{year}_{industry_level}.csv"
        csv_file_path = os.path.join(geo_path, csv_file_name)

        # Fetch data for each combination
        cursor.execute("""
        SELECT GeoID, Establishments, Employees, Payroll FROM DataEntry
        WHERE GeoID = ? AND Year = ? AND IndustryLevel = ?
        """, (geo_id, year, industry_level))
        data_entries = cursor.fetchall()

        # Write data to the CSV file
        with open(csv_file_path, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["Zip", "Establishments", "Employees", "Payroll"])
            writer.writerows(data_entries)

# Close the database connection
cursor.close()
conn.close()


KeyboardInterrupt: 